In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras.utils import np_utils
import coremltools

Using TensorFlow backend.


In [2]:
def mnist_data():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    
    Y_train = np_utils.to_categorical(y_train, 10)
    Y_test = np_utils.to_categorical(y_test, 10)
    
    X_train = X_train.reshape([-1, 28, 28, 1])
    X_test  = X_test.reshape([-1, 28, 28, 1])
    return (X_train, Y_train), (X_test, Y_test)

In [3]:
def network():
    model = Sequential()

    model.add(Flatten(input_shape=(28, 28, 1, )))
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    return model

In [10]:
(X_train, Y_train), (X_test, Y_test) = mnist_data()

model = network()
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=100, nb_epoch=1)

score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
flatten_2 (Flatten)              (None, 784)           0           flatten_input_2[0][0]            
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 256)           200960      flatten_2[0][0]                  
____________________________________________________________________________________________________
batchnormalization_3 (BatchNorma (None, 256)           1024        dense_4[0][0]                    
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 256)           0           batchnormalization_3[0][0]       
___________________________________________________________________________________________

In [11]:
coreml_model = coremltools.converters.keras.convert(model,
                                                    input_names=['input'],
                                                    output_names=['probs'],
                                                    image_input_names='input',
                                                    class_labels=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
                                                    predicted_feature_name='predictedNumber')

0 : flatten_input_2, <keras.engine.topology.InputLayer object at 0x114116390>
1 : flatten_2, <keras.layers.core.Flatten object at 0x114116350>
2 : dense_4, <keras.layers.core.Dense object at 0x11411a210>
3 : batchnormalization_3, <keras.layers.normalization.BatchNormalization object at 0x11412bdd0>
4 : activation_4, <keras.layers.core.Activation object at 0x114116650>
5 : dense_5, <keras.layers.core.Dense object at 0x11243fcd0>
6 : batchnormalization_4, <keras.layers.normalization.BatchNormalization object at 0x111b41350>
7 : activation_5, <keras.layers.core.Activation object at 0x1123eadd0>
8 : dense_6, <keras.layers.core.Dense object at 0x1122b89d0>
9 : activation_6, <keras.layers.core.Activation object at 0x1124ee850>


In [12]:
coreml_model.author = 'ytakzk'
coreml_model.license = 'MIT'
coreml_model.short_description = 'MNIST handwriting recognition by 3-layered neural network'
coreml_model.input_description['input'] = '28x28 grayscaled pixel values between 0-1'
coreml_model.output_description['probs'] = '10-digit probabilities'
coreml_model.output_description['predictedNumber'] = 'predicted number bewteen 0-9'
coreml_model.save('SimpleMnist.mlmodel')

print(coreml_model)

input {
  name: "input"
  shortDescription: "28x28 grayscaled pixel values between 0-1"
  type {
    imageType {
      width: 28
      height: 28
      colorSpace: GRAYSCALE
    }
  }
}
output {
  name: "probs"
  shortDescription: "10-digit probabilities"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "predictedNumber"
  shortDescription: "predicted number bewteen 0-9"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "predictedNumber"
predictedProbabilitiesName: "probs"
metadata {
  shortDescription: "MNIST handwriting recognition by 3-layered neural network"
  author: "ytakzk"
  license: "MIT"
}



In [13]:
model =  coremltools.models.MLModel('SimpleMnist.mlmodel')

(X_train, _), _ = mnist_data()
predictions = model.predict({'input': X_train[0]})
print(predictions)

Exception: Model prediction is only supported on macOS version 10.13.